In [69]:
import pandas as pd
import nb_utils
import html
import os
import requests

pd.set_option("mode.copy_on_write", True)

In [70]:
nb_utils.set_path()
from tomato_stream import utils

In [71]:
API_KEY = os.environ["API_KEY"]

In [72]:
API_KEY

'b59a7e05'

In [73]:
url = "https://unogs-unogs-v1.p.rapidapi.com/search/titles"

params = {
	"limit": 200000,
	"country_list": 39,
}

headers = {
	"x-rapidapi-key": "d2f42f66abmsh88fcea3d1b94b1ap1aa687jsna81b8ab76c56",
	"x-rapidapi-host": "unogs-unogs-v1.p.rapidapi.com",
}

response = requests.get(url, headers=headers, params=params)

results = response.json()["results"]

In [74]:
catalog = (
	pd.DataFrame(results)
	.loc[lambda _df: _df["imdb_id"] != ""]
	.assign(
		title=lambda _df: _df["title"].apply(html.unescape),
		year=lambda _df: _df["year"].astype(int),
		link=lambda _df: _df["netflix_id"].apply(
			lambda x: f"https://www.netflix.com/title/{x}"
		),
	)
	.loc[:, ["title", "title_type", "imdb_id", "link"]]
)

In [75]:
catalog

,title,title_type,imdb_id,link
0,Para Betina Pengikut Iblis Part 2,movie,tt31418314,https://www.netflix.com/title/81927386
1,Chances Are You and I,movie,tt32306491,https://www.netflix.com/title/81765097
2,What's Next? The Future with Bill Gates,series,tt0207201,https://www.netflix.com/title/81609333
3,Envious,series,tt4869308,https://www.netflix.com/title/81687655
4,Live from the Other Side with Tyler Henry,series,tt33096473,https://www.netflix.com/title/81786596
...,...,...,...,...
7539,3:10 to Yuma,movie,tt0381849,https://www.netflix.com/title/70065114
7540,Kill Bill: Vol. 1,movie,tt0266697,https://www.netflix.com/title/60031236
7541,50 First Dates,movie,tt0343660,https://www.netflix.com/title/60033311
7542,Year One,movie,tt1045778,https://www.netflix.com/title/70112729


In [78]:
def get_rating(imdb_id):
	params = {"apikey": API_KEY, "i": imdb_id}
	url = "http://www.omdbapi.com/"
	response = requests.get(url, params=params)
	if response.status_code == 200:
		response_json = response.json()

		title = response_json.get("Title", "")
		year = response_json.get("Year", "")
		genres = response_json.get("Genre", "")
		runtime = response_json.get("Runtime", "")
		actors = response_json.get("Actors", "")
		director = (response_json.get("Director", ""),)
		plot = response_json.get("Plot", "")
		poster = response_json.get("Poster", "")

		rotten_tomatoes_rating = ""

		if "Ratings" in response_json:
			for rating in response_json["Ratings"]:
				if rating["Source"] == "Rotten Tomatoes":
					rotten_tomatoes_rating = int(rating["Value"].strip("%"))
					break

		return (
			title,
			year,
			genres,
			runtime,
			actors,
			director,
			plot,
			rotten_tomatoes_rating,
			poster,
		)
	else:
		raise Exception(f"Error: {response.status_code}")

In [79]:
catalog[
	[
		"title_2",
		"year",
		"genres",
		"runtime",
		"actors",
		"director",
		"plot",
		"rating",
		"poster",
	]
] = catalog["imdb_id"].apply(get_rating).apply(pd.Series)

In [80]:
catalog

,title,title_type,imdb_id,link,title_2,year,genres,runtime,actors,director,plot,rating,poster
0,Para Betina Pengikut Iblis Part 2,movie,tt31418314,https://www.netflix.com/title/81927386,Para Betina Pengikut Iblis: Part 2,2024,"Horror, Thriller",90 min,"Adipati Dolken, Mawar Eva de Jongh, Hanggini","(Rako Prijanto,)",Kiai Taqim had heard about the chaos that occu...,,https://m.media-amazon.com/images/M/MV5BNGRkOW...
1,Chances Are You and I,movie,tt32306491,https://www.netflix.com/title/81765097,"Chances Are, You and I",2024,"Comedy, Drama, Romance",133 min,"Kelvin Miranda, Kira Balinger, JinHo Bae","(Cathy Camarillo,)","Sol Sikat, burdened by a brain tumor and pessi...",,https://m.media-amazon.com/images/M/MV5BYTg2NG...
2,What's Next? The Future with Bill Gates,series,tt0207201,https://www.netflix.com/title/81609333,What Women Want,2000,"Comedy, Fantasy, Romance",127 min,"Mel Gibson, Helen Hunt, Marisa Tomei","(Nancy Meyers,)","A cocky, chauvinistic advertising executive ma...",54,https://m.media-amazon.com/images/M/MV5BZjUyZW...
3,Envious,series,tt4869308,https://www.netflix.com/title/81687655,Envious,2016,"Short, Drama, Thriller",11 min,"April Hartman, Sheril Rodgers, Matthew Roy","(Roger Sampson,)",A monthly family dinner goes horribly wrong wh...,,https://m.media-amazon.com/images/M/MV5BNWQ1Mz...
4,Live from the Other Side with Tyler Henry,series,tt33096473,https://www.netflix.com/title/81786596,Live from the Other Side with Tyler Henry,2024–,Talk-Show,N/A,Tyler Henry,"(N/A,)","Celebrity guests connect with Henry, ""America'...",,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7539,3:10 to Yuma,movie,tt0381849,https://www.netflix.com/title/70065114,3:10 to Yuma,2007,"Action, Crime, Drama",122 min,"Russell Crowe, Christian Bale, Ben Foster","(James Mangold,)",A small-time rancher agrees to hold a captured...,89,https://m.media-amazon.com/images/M/MV5BODE0NT...
7540,Kill Bill: Vol. 1,movie,tt0266697,https://www.netflix.com/title/60031236,Kill Bill: Vol. 1,2003,"Action, Crime, Thriller",111 min,"Uma Thurman, David Carradine, Daryl Hannah","(Quentin Tarantino,)","After waking from a 4-year coma, a former assa...",85,https://m.media-amazon.com/images/M/MV5BNzM3ND...
7541,50 First Dates,movie,tt0343660,https://www.netflix.com/title/60033311,50 First Dates,2004,"Comedy, Drama, Romance",99 min,"Adam Sandler, Drew Barrymore, Rob Schneider","(Peter Segal,)",Henry Roth is a man afraid of commitment until...,45,https://m.media-amazon.com/images/M/MV5BMjAwMz...
7542,Year One,movie,tt1045778,https://www.netflix.com/title/70112729,Year One,2009,Comedy,97 min,"Jack Black, Michael Cera, Olivia Wilde","(Harold Ramis,)","After being banished from their tribe, two hun...",14,https://m.media-amazon.com/images/M/MV5BMDkyMT...


In [81]:
params = {"apikey": API_KEY, "i": "tt10855768"}
url = "http://www.omdbapi.com/"
requests.get(url, params=params).json()

{'Title': 'Missing',
 'Year': '2023',
 'Rated': 'PG-13',
 'Released': '20 Jan 2023',
 'Runtime': '111 min',
 'Genre': 'Drama, Mystery, Thriller',
 'Director': 'Nicholas D. Johnson, Will Merrick',
 'Writer': 'Will Merrick, Nicholas D. Johnson, Sev Ohanian',
 'Actors': 'Tim Griffin, Ava Zaria Lee, Nia Long',
 'Plot': "When her mother disappears while on vacation in Colombia with her new boyfriend, June's search for answers is hindered by international red tape. However, as she digs deeper, her digital sleuthing soon raises more questions than a...",
 'Language': 'English, Spanish',
 'Country': 'United States, Russia',
 'Awards': '3 wins & 1 nomination',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjAzYjUwNWUtYTYxOS00MTkzLWJkOTUtOTE4MTE4ODRmNGNhXkEyXkFqcGdeQXVyMTA3MDk2NDg2._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.1/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '88%'},
  {'Source': 'Metacritic', 'Value': '66/100'}],
 'Metascore': '66',
 'imdbR

In [82]:
ratings_df = (
	catalog.loc[(catalog.title == catalog.title_2) & (catalog.rating != "")]
	.drop(columns=["title_2", "imdb_id"])
	.reset_index(drop=True)
	.rename(
		columns={
			"title": "Titel",
			"title_type": "Typ",
			"year": "Jahr",
			"link": "Link",
			"genres": "Genres",
			"runtime": "Dauer",
			"actors": "Schauspieler",
			"director": "Regisseur",
			"plot": "Handlung",
			"rating": "Tomatorscore",
			"poster": "Poster",
		}
	)
)

In [83]:
ratings_df

,Titel,Typ,Link,Jahr,Genres,Dauer,Schauspieler,Regisseur,Handlung,Tomatorscore,Poster
0,Missing,movie,https://www.netflix.com/title/81720826,2023,"Drama, Mystery, Thriller",111 min,"Tim Griffin, Ava Zaria Lee, Nia Long","(Nicholas D. Johnson, Will Merrick,)",When her mother disappears while on vacation i...,88,https://m.media-amazon.com/images/M/MV5BMjAzYj...
1,Uglies,movie,https://www.netflix.com/title/81002266,2024,"Action, Adventure, Drama",100 min,"Joey King, Chase Stokes, Laverne Cox","(McG,)",A world in which a compulsory operation wipes ...,20,https://m.media-amazon.com/images/M/MV5BOGEyYm...
2,Rebel Ridge,movie,https://www.netflix.com/title/81157729,2024,"Action, Crime, Drama",131 min,"Aaron Pierre, Don Johnson, AnnaSophia Robb","(Jeremy Saulnier,)",An ex-Marine grapples his way through a web of...,95,https://m.media-amazon.com/images/M/MV5BYTE4ZD...
3,The Deliverance,movie,https://www.netflix.com/title/81582382,2024,"Horror, Thriller",N/A,"Andra Day, Glenn Close, Anthony B. Jenkins","(Lee Daniels,)",A family living in a home in Indiana discover ...,35,https://m.media-amazon.com/images/M/MV5BNWI2ZW...
4,Breathless,series,https://www.netflix.com/title/81667161,1960,"Crime, Drama",90 min,"Jean-Paul Belmondo, Jean Seberg, Van Doude","(Jean-Luc Godard,)","A small-time crook, hunted by the authorities ...",95,https://m.media-amazon.com/images/M/MV5BNTAyY2...
...,...,...,...,...,...,...,...,...,...,...,...
1524,Kill Bill: Vol. 2,movie,https://www.netflix.com/title/60032563,2004,"Action, Crime, Thriller",137 min,"Uma Thurman, David Carradine, Michael Madsen","(Quentin Tarantino,)",The Bride continues her quest of vengeance aga...,84,https://m.media-amazon.com/images/M/MV5BNmFiYm...
1525,3:10 to Yuma,movie,https://www.netflix.com/title/70065114,2007,"Action, Crime, Drama",122 min,"Russell Crowe, Christian Bale, Ben Foster","(James Mangold,)",A small-time rancher agrees to hold a captured...,89,https://m.media-amazon.com/images/M/MV5BODE0NT...
1526,Kill Bill: Vol. 1,movie,https://www.netflix.com/title/60031236,2003,"Action, Crime, Thriller",111 min,"Uma Thurman, David Carradine, Daryl Hannah","(Quentin Tarantino,)","After waking from a 4-year coma, a former assa...",85,https://m.media-amazon.com/images/M/MV5BNzM3ND...
1527,50 First Dates,movie,https://www.netflix.com/title/60033311,2004,"Comedy, Drama, Romance",99 min,"Adam Sandler, Drew Barrymore, Rob Schneider","(Peter Segal,)",Henry Roth is a man afraid of commitment until...,45,https://m.media-amazon.com/images/M/MV5BMjAwMz...


In [84]:
ratings_df.to_pickle(utils.get_output_path("ratings.pkl"))